# Einsums

In [2]:
import numpy as np 
from numpy.random import seed, randint

http://ajcr.net/Basic-guide-to-einsum/

In [20]:
seed(1643)
A = randint(0, 10, 4).reshape(2, 2)
A

array([[5, 9],
       [6, 2]])

$\sum_{i} A_{i,i}$

In [21]:
# Sum over the diagonal (the trace)
np.einsum("ii->", A)

7

In [22]:
# Sum all of the elements of A
np.einsum("ij->", A)

22

In [24]:
# ∀ col, sum over rows
np.einsum("ij->j", A)

array([11, 11])

In [31]:
# ∀ row, sum over columns
np.einsum("ij->i", A)

array([14,  8])

In [33]:
# Transpose a matrix
np.einsum("ij->ji", A)

array([[5, 6],
       [9, 2]])

$\sum_{i} a_i  b_i$

In [41]:
a, b = np.array([1,2,3]), np.array([1, 1, 1])

In [42]:
# Dot product
np.einsum("i,i->", a, b)

6

In [48]:
# Sum over 'b' and multiply by each entry of i
np.einsum("i,j->i", a,b)

array([3, 6, 9])

In [63]:
# The outer-product of 'a'
np.einsum("i,j->ij", a, a)

array([[1, 2, 3],
       [2, 4, 6],
       [3, 6, 9]])

## Matrix Multiplication

$(AB)_{i,k} = A_{i,:}\cdot B_{:,k} = \sum_{j=1}^n A_{i,j} B_{j, k}$

In [8]:
A = np.array([[1, 1, 1],
              [2, 2, 2],
              [5, 5, 5]])

B = np.array([[0, 1, 0],
              [1, 1, 0],
              [1, 1, 1]])

In [10]:
np.einsum("ij,jk -> ik", A, B)

array([[ 2,  3,  1],
       [ 4,  6,  2],
       [10, 15,  5]])

## In Higher Dimensions
Given the following 5x3 matrix of integers, suppose we want to compute the outer-product of each column.
The resulting array should be of shape `(3, 5, 5)`

In [68]:
seed(314)
Phi = np.random.randint(0, 10, (5, 3))
Phi

array([[8, 9, 3],
       [6, 0, 7],
       [2, 8, 6],
       [7, 0, 3],
       [6, 7, 8]])

### For a single column

In matrix notation, the outer-product of a vector is given by the product $aa^T$

In [71]:
Phi[:, [0]] @ Phi[:, [0]].T

array([[64, 48, 16, 56, 48],
       [48, 36, 12, 42, 36],
       [16, 12,  4, 14, 12],
       [56, 42, 14, 49, 42],
       [48, 36, 12, 42, 36]])

Under `einsum`, we can express the outer-product of any two unidimensional arrays $a$, $b$

In [161]:
vPhi_i = Phi[:, 0]
np.einsum("i,j->ij", vPhi_i, vPhi_i)

array([[64, 48, 16, 56, 48],
       [48, 36, 12, 42, 36],
       [16, 12,  4, 14, 12],
       [56, 42, 14, 49, 42],
       [48, 36, 12, 42, 36]])

In [158]:
vPhi_i = Phi[:, [0]]
np.einsum("ik,jk->kij", vPhi_i, vPhi_i)

array([[[64, 48, 16, 56, 48],
        [48, 36, 12, 42, 36],
        [16, 12,  4, 14, 12],
        [56, 42, 14, 49, 42],
        [48, 36, 12, 42, 36]]])

### For every column

In [159]:
# For those that have the same column 'j', compute the
# matrix resulting in evaluating every pair of value multiplication
# (i,j)
np.einsum("ik,jk->kij", Phi, Phi)

array([[[64, 48, 16, 56, 48],
        [48, 36, 12, 42, 36],
        [16, 12,  4, 14, 12],
        [56, 42, 14, 49, 42],
        [48, 36, 12, 42, 36]],

       [[81,  0, 72,  0, 63],
        [ 0,  0,  0,  0,  0],
        [72,  0, 64,  0, 56],
        [ 0,  0,  0,  0,  0],
        [63,  0, 56,  0, 49]],

       [[ 9, 21, 18,  9, 24],
        [21, 49, 42, 21, 56],
        [18, 42, 36, 18, 48],
        [ 9, 21, 18,  9, 24],
        [24, 56, 48, 24, 64]]])